# 自回归分布滞后模型

## 原理讲解

在自回归模型中，也可以引入其他变量来构成“自回归分布滞后模型”（Autoregressive Distributed Lag Model，简记 $ADL(p,q)$：

$$y_{t}=\beta_{0}+\beta_{1} y_{t-1}+\cdots+\beta_{p} y_{t-p}+\gamma_{1} x_{t-1}+\cdots+\gamma_{q} x_{t-q}+\varepsilon_{t}\tag{1}$$

如果自回归分布滞后模型满足以下假定则万事大吉，可以用OLS来估计它。
1. $\mathrm{E}\left(\varepsilon_{t} | y_{t-1}, y_{t-2}, \cdots, x_{1, t-1}, x_{1, t-2}, \cdots, x_{K, t-1}, x_{K, t-2}, \cdots\right)=0$。这个假定类似于严格外生性假设，它意味着扰动项 $\varepsilon_{t}$ 与所有解释变量的整个历史全部无关。这保证了对滞后期数 $\left(p, q_{1}, \cdots, q_{K}\right)$ 的设定是正确的。如果滞后期数的设定不正确，比如，真实模型还应该包括 $y_{t-（p+1）}$，但该项 $\beta_{p+1} y_{t-(p+1)}$ 却被纳入扰动项中，则扰动项 $\varepsilon_{t}$ 便与解释变量相关，导致OLS不致。
2. $\left\{y_{t}, x_{1 t}, \cdots, x_{K t}\right\}$ 为渐近独立的平稳序列。
3. $\left\{y_{t}, x_{1 t}, \cdots, x_{K t}\right\}$ 有非零的有限四阶矩。
4. 解释变量无完全多重共线性。

对滞后期数的选择可以使用信息准则（最小化 AIC 或 BIC），或使用 $t,F$ 检验来检验最后期系数的显著性。更一般地，可以在 $ARMA$ 模型中引入其他变量，称为“$ARMAX$”模型。

In [1]:
import pandas as pd
import statsmodels.api as sm

data = pd.read_excel('../数据/上证指数与沪深300.xlsx')
Y = data['hs300']
X = data['sz']

def lag_list(Y, X, p=1, q=1):
    '''
    待估计方程：y = c + y(-1) +....+y(-p) + x(-1) + ... + x(-q)
    获取自回归分布滞后模型的估计向量

    Parameters
    ----------
    Y : 被估计变量
    X : 估计变量
    p : ADL 模型 Y 的滞后阶数，标量默认为1
    q : ADL 模型 X 的滞后阶数，标量默认为1

    Returns
    -------
    ADLy : ADL 模型被解释变量
    ADLx : ADL 模型解释变量

    '''
    ADLx = pd.DataFrame()
    T = len(Y)
    ADLy = list(Y[max(p, q):T])
    for i in range(1, p+1):
        name = f'y_{i}'
        ADLx[name] = list(Y[max(p, q)-i:T-i])
    for i in range(1, q+1):
        name = f'x_{i}'
        ADLx[name] = list(X[max(p, q)-i:T-i])
    return ADLy, ADLx
    
ADLy, ADLx = lag_list(Y, X ,p=2, q=2)
ADLx = sm.add_constant(ADLx)
mod = sm.OLS(ADLy, ADLx)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.980
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     5540.
Date:                Fri, 29 May 2020   Prob (F-statistic):               0.00
Time:                        19:37:29   Log-Likelihood:                -2414.2
No. Observations:                 458   AIC:                             4838.
Df Residuals:                     453   BIC:                             4859.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         53.2565     26.908      1.979      0.048       0.376     106.137
y_1            1.7194      0.202      8.511      0.000       1.322       2.116
y_2           -0.7140      0.203     -3.517      0.000      -1.113      -0.315
x_1           -1.0193      0.274     -3.727      0.000      -1.557      -0.482
x_2            0.9939      0.275      3.620      0.000       0.454       1.534
==============================================================================
Omnibus:                       28.071   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               85.486
Skew:                          -0.167   Prob(JB):                     2.73e-19
Kurtosis:                       5.090   Cond. No.                     8.10e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.1e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## statsmodels 库实现

未找到，官方介绍基本模型只有 AR、VAR、ARMA

statsmodels.tsa contains model classes and functions that are useful for time series analysis. Basic models include univariate autoregressive models (AR), vector autoregressive models (VAR) and univariate autoregressive moving average models (ARMA). Non-linear models include Markov switching dynamic regression and autoregression. It also includes descriptive statistics for time series, for example autocorrelation, partial autocorrelation function and periodogram, as well as the corresponding theoretical properties of ARMA or related processes. It also includes methods to work with autoregressive and moving average lag-polynomials. Additionally, related statistical tests and some useful helper functions are available.

## matlab 实现

ADLxx.m
```matlab
function [ADLy,ADLx] = ADLxx(Y,X,p,q)

% 待估计方程：y = c + y(-1) +....+y(-p) + x(-1) + ... + x(-q)
% 获取自回归分布滞后模型的估计向量
%  X   - 解释变量，列向量
%  Y   - 被解释变量，列向量
%  p   - ADL模型Y的滞后阶数,标量
%  q   - ADL模型X的滞后阶数,标量

T = length(Y);
N = max(p,q)+1;

ADLy = Y(N:T);
c = ones(length(ADLy),1);  %常数项
ADLx = zeros(length(ADLy),1+p+q);
ADLx(:,1) = c;
for i=1:p, ADLx(:,1+i) = Y(N-i:T-i);end
for i=1:q, ADLx(:,1+p+i) = X(N-i:T-i);end

% % 1.载入数据
% data = xlsread('../数据/上证指数与沪深300.xlsx');
% f1 = data(:,1); f2 = data(:,2)
% 
% % 2.估计ADL模型f1 = c + f1(-1) + f1(-2) + e(-1)
% [ADLy,ADLx] = ADLxx(f1,e,2,1);
% beta = regress(ADLy,ADLx);
```